In [ ]:
!pip install -q transformers

     |████████████████████████████████| 5.5 MB 7.2 MB/s 
     |████████████████████████████████| 7.6 MB 46.6 MB/s 
     |████████████████████████████████| 163 kB 67.5 MB/s 


In [ ]:
!pip install -q youtube_transcript_api

In [ ]:
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi


In [ ]:
youtube_vid="https://www.youtube.com/watch?v=YereI6Gn3bM&t=514s"

In [ ]:
vid_id=youtube_vid.split("=")[1]

In [ ]:
vid_id

'YereI6Gn3bM&t'

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo(vid_id)

In [ ]:
YouTubeTranscriptApi.get_transcript(vid_id)
transcript=YouTubeTranscriptApi.get_transcript(vid_id)

In [ ]:
transcript[0:5]

[{'text': "speech it's the most natural form of",
  'start': 0.64,
  'duration': 4.88},
 {'text': 'human communication', 'start': 3.76, 'duration': 4.24},
 {'text': 'this is my demo of a real-time speech',
  'start': 5.52,
  'duration': 3.68},
 {'text': 'recognition system', 'start': 8.0, 'duration': 4.32},
 {'text': 'using deep learning', 'start': 9.2, 'duration': 3.12}]

In [ ]:
result=""
for i in transcript:
  result+=' '+i['text']
print(len(result))

14958


In [ ]:
summarizer=pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
 num_iters=int(len(result)/1000)
summarized_text=[]
for i in range(0,num_iters+1):
  start=0
  start=i*1000
  end=(i+1)*1000
  out=summarizer(result[start:end])
  out=out[0]
  out=out['summary_text']
  summarized_text.append(out)
print(summarized_text)

[" A real-time speech recognition system using deep learning can help solve the problem of speech recognition . We're going to build our own neural network and train the speech recognition model from scratch . This is a demo of a speech recognition using deep-learned neural networks . We'll show how deep-learning can be used to solve this problem in this video .", ' Language itself is really complex and it has a lot of nuance and variations that you would have to come up with all possible rules for it as well to have an effective speech recognition system . So what do you do when you have like what seems like an insurmountable amount of rules you use deep learning .', ' The acoustic model will be a neural network that takes in speech waves as input and outputs to transcribe text . The neural network also needs to be lightweight in terms of memory and compute because we want to run it real time on everyday consumer machines . Recurrent neural networks or rnn are a natural fit for this t

In [ ]:
 print(len(summarized_text))

15


In [ ]:
summarized_text

[" A real-time speech recognition system using deep learning can help solve the problem of speech recognition . We're going to build our own neural network and train the speech recognition model from scratch . This is a demo of a speech recognition using deep-learned neural networks . We'll show how deep-learning can be used to solve this problem in this video .",
 ' Language itself is really complex and it has a lot of nuance and variations that you would have to come up with all possible rules for it as well to have an effective speech recognition system . So what do you do when you have like what seems like an insurmountable amount of rules you use deep learning .',
 ' The acoustic model will be a neural network that takes in speech waves as input and outputs to transcribe text . The neural network also needs to be lightweight in terms of memory and compute because we want to run it real time on everyday consumer machines . Recurrent neural networks or rnn are a natural fit for this

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
nlp=spacy.load('en_core_web_sm')


In [ ]:
def summarize_sentiment(text):
    stopwords=list(STOP_WORDS)
    nlp=spacy.load('en_core_web_sm')
    doc=nlp(text)
    tokens=[token.text for token in doc]
    word_freq={}
    for word in doc:
        if word.text.lower() not in stopwords:
            if word.text.lower() not in punctuation:
                if word.text not in word_freq.keys():
                    word_freq[word.text]=1
                else:
                    word_freq[word.text]+=1
    max_freq=max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word]=word_freq[word]/max_freq
    sentence_tok=[sent for sent in doc.sents]
    sent_scores={}
    for sent in sentence_tok:
        for word in sent:
            if word.text.lower() in word_freq.keys():
                if sent not in sent_scores.keys():
                    sent_scores[sent]=word_freq[word.text.lower()]
                else:
                    sent_scores[sent]+=word_freq[word.text.lower()]
                    
                
    select_len=int(len(sentence_tok)*0.22)
    summary=nlargest(select_len,sent_scores,key=sent_scores.get)
    final_sum=[word.text for word in summary]
    summary=' '.join(final_sum)
    return(summary)
    
    

In [ ]:

vid_sum=summarize_sentiment(str(summarized_text))

In [ ]:
import torch

In [ ]:
from transformers import BertForQuestionAnswering
model=BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
question="what will n layer do?"
answer="n layer will produce features that are more robust"

In [ ]:
input_ids = tokenizer.encode(question, answer)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 18 tokens.


In [ ]:
sep_index=input_ids.index(tokenizer.sep_token_id)
seg_a=sep_index+1
seg_b=len(input_ids)-seg_a
seg_ids=[0]*seg_a+[1]*seg_b
assert len(seg_ids)==len(input_ids)

In [ ]:
output=model(torch.tensor([input_ids]),torch.tensor([seg_ids]),return_dict=True)
start_score=output.start_logits
end_score=output.end_logits

In [ ]:
def answer_question(question, answer_text):
    input_ids = tokenizer.encode(question, answer_text)
    print('Query has {:,} tokens.\n'.format(len(input_ids)))
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    outputs = model(torch.tensor([input_ids]),token_type_ids=torch.tensor([segment_ids]),return_dict=True) 
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]
    print('Answer: "' + answer + '"')

In [ ]:
import textwrap
wrapper = textwrap.TextWrapper(width=80) 
bert_abstract = summarize_sentiment(str(summarized_text))
print(wrapper.fill(bert_abstract))

It learns to extract better features from the mel spectrogram and also reduce
the time dimensions of the data i.i.  need to process our character labels our
models will be character based meaning it will output characters instead of word
probabilities decoding character probabilities is more efficient .', ' n theory
they say an n layer will produce features that should be more robust causing the
rnns to produce better predictions we also set the shot of the cnn layer to 2
therefore reducing the time steps of the male spectrogram by half allowing the
rns to do less work because there are less time steps which would make the
overall network faster .', ' The code is open source Recurrent neural networks
or rnn are a natural fit for this task as it can process sequential data .', " A
neural network can learn is dependent on the data you train it with if we want
our neural network to learn the nuances of speech we'll need speech data that
has a lot of variations of gender, age accent, envir

In [ ]:
question = "What does neural network do?"

answer_question(question, bert_abstract)

Query has 498 tokens.

Answer: "takes in speech waves as input and outputs to transcribe text"


In [ ]:
question="why do we use deep learning?"
answer_question(question,bert_abstract)

Query has 499 tokens.

Answer: "if you don ' t have a gpi you can use free alternatives like google collab or kaggle kernels . so what do you do when you have like what seems like an insurmountable amount of rules"


In [ ]:
question = "What did baidu do?"

answer_question(question, bert_abstract)

Query has 498 tokens.

Answer: "speech recognition paper from baidu claimed that you need about 11 000 hours of audio data to have an effective speech recognition system"


In [ ]:
question = "What are the variations in speech text?"

answer_question(question, bert_abstract)

Query has 500 tokens.

Answer: "gender , age accent , environmental noise and types of microphones"


In [ ]:
question = "how much hours of audio required for effective speech recognition ?"

answer_question(question, bert_abstract)

Query has 503 tokens.

Answer: "11 000 hours"


In [ ]:
question = "What is this speech is about ?"

answer_question(question, bert_abstract)

Query has 499 tokens.

Answer: "[CLS]"
